In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
#Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML Lab 4111/diabetes.csv')

In [4]:
#Features Selection
X = dataset #include all columns
Y = dataset[['Outcome']] #target variable; only Outcome column
X = X.drop(['Outcome'], axis = 1) #remove Outcome from X, containing only Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age

In [5]:
#Split the dataset into training-test set
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [6]:
#Check shape
print(X.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(768, 8)
(614, 8)
(614, 1)
(154, 8)
(154, 1)


## 2 Hidden layers Neural Network Initialization

In [7]:
#Initialization of Parameters
def parameter_initialization(nx,nh,ny):

  #Random initialization of weights matrix of shape layers_dimension[l] and biases vector of shape layers-dimension[l-1]
  W1 = np.random.randn(nh, nx)*0.01
  b1 = np.zeros((nh,1))
  W2 = np.random.randn(nh, nh)*0.01
  b2 = np.zeros((nh,1))
  W3 = np.random.randn(ny, nh)*0.01
  b3 = np.zeros((ny,1))

  return {"W1": W1, "b1": b1, "W2": W2, "b2": b2, "W3": W3, "b3": b3}

## Forward Propagation

In [8]:
#Activation function - sigmoid
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

#Forward Propagation 
def forward_propagation(X, parameters):
  
    #Formul z = wx + b; A = sigmoid(z)
    Z1 = np.dot(parameters['W1'], X) + parameters['b1']
    A1 = sigmoid(Z1)

    Z2 = np.dot(parameters['W2'], A1) + parameters['b2']
    A2 = sigmoid(Z2)
    
    Z3 = np.dot(parameters['W3'], A2) + parameters['b3']
    A3 = sigmoid(Z3)

    return {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2, "Z3": Z3, "A3": A3} 

## Cost Calculation

In [9]:
def cost_estimation(true, predicted):
  #formula of cost = -(∑(ylogy^ + (1-y)log(1-y^)))/m
  log_probs = np.multiply(np.log(predicted), true) + np.multiply(np.log(1 - predicted), (1 - true))
  total_cost = -np.sum(log_probs)/true.shape[1]

  return np.squeeze(total_cost)

## Backward Propagation

In [10]:
#Backward Propagation function
def backward_propagation(parameters, activations, X, Y):
  '''
  dwl = dzl*(A[l-1].T) /m
  dbl = sum(dzli)/m
  dzl = dAl*g'(zl)
  '''
  m = X.shape[1]

  #Output layer
  dZ3 = activations['A3'] - Y                  #Computed the error derivative 
  dW3 = np.dot(dZ3, activations['A2'].T) / m   #Computed the weight derivative 
  db3 = np.sum(dZ3, axis=1, keepdims = True)/m #Computed the bias derivative - keepdims = True to keep the input-output dimensions same
  
  #Hidden layer 2
  dZ2 = np.dot(parameters['W3'].T, dZ3)*(activations['A2']-np.power(activations['A2'], 2))
  dW2 = np.dot(dZ2, activations['A1'].T)/m
  db2 = np.sum(dZ2, axis=1,keepdims=True)/m
  
  #Hidden layer 1
  dZ1 = np.dot(parameters['W2'].T, dZ2)*(activations['A1']-np.power(activations['A1'], 2))
  dW1 = np.dot(dZ1, X.T)/m
  db1 = np.sum(dZ1, axis=1,keepdims=True)/m
  
  return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2, "dW3": dW3, "db3": db3}

def parameters_update(parameters, gradients, alpha = 0.01):
  #parameters update rules w = w - learning_rate * dw ; b = b - learning_rate * db
  parameters['W1'] = parameters['W1'] - alpha * gradients['dW1']
  parameters['b1'] = parameters['b1'] - alpha * gradients['db1']
  parameters['W2'] = parameters['W2'] - alpha * gradients['dW2']
  parameters['b2'] = parameters['b2'] - alpha * gradients['db2']
  parameters['W3'] = parameters['W3'] - alpha * gradients['dW3']
  parameters['b3'] = parameters['b3'] - alpha * gradients['db3']

  return parameters

# Neural Network Model

In [11]:
def neural_network_model(X,Y, n_h = 10, epochs = 100):
  n_x = X.shape[0]     #Number of nodes/neurons in input layer
  n_y = Y.shape[0]     #Number of nodes/neurons in output layer

  parameters = parameter_initialization(n_x, n_h, n_y)
  for i in range(0,epochs):
    results = forward_propagation(X, parameters)
    errors = cost_estimation(Y, results['A3'])
    grads = backward_propagation(parameters, results, X,Y)
    params = parameters_update(parameters, grads)

  return parameters

In [12]:
x = X_train.T   #(614,8) -> (8,614)
y = y_train.values.reshape(1, y_train.size) #(614,1) -> (1,614)
model = neural_network_model(x,y, n_h = 100, epochs = 100)

# Perforane Evaluation

In [13]:
def predict(parameters, X):
    results = forward_propagation(X, parameters)
    #print (results['A3'][0])
    predictions = np.around(results['A3'])    
    return predictions

predictions = predict(model, x)
print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

Accuracy: 65%
